In [1]:
!pip install transformers sentencepiece --quiet

     |████████████████████████████████| 5.5 MB 27.3 MB/s 
     |████████████████████████████████| 1.3 MB 52.9 MB/s 
     |████████████████████████████████| 7.6 MB 61.7 MB/s 
     |████████████████████████████████| 163 kB 72.2 MB/s 


In [2]:
!pip install datasets --quiet

     |████████████████████████████████| 441 kB 34.0 MB/s 
     |████████████████████████████████| 115 kB 55.2 MB/s 
     |████████████████████████████████| 95 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 68.3 MB/s 
     |████████████████████████████████| 127 kB 49.4 MB/s 
     |████████████████████████████████| 115 kB 67.5 MB/s 


In [3]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-3b40850a8ccdf74d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [5]:
corpus['train'][0:9]

{'id': ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем'],
  ['В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото)',
   'Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства',
   'В Индии женщины встали живой стеной длиной 620 км'],
  ['Томас Бах: "Олимпийский комитет России понес достаточное наказание"',
   'Президент МОК Бах: Россия уже достаточно наказана за допинг',
   'Глава МОК заявил, что ОКР понес наказание за манипуляции с пробами на ОИ в Сочи'],
  ['Россия не представила WADA данные по пробам в срок и может быть наказана',
   'WADA: Россия не предоставила нам доступ к допинг-пробам',
   'В WADA пожаловались на отсутствие доступа к допинг-пробам российских спортсменов'],
  ['Антидопинговое агентство США призвало ВАДА отстранить российских спортсменов от соревнован

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [7]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
        
    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"
        
    return [text1, text2, p[0], p[1], paraphrase]

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name_gen = 'cointegrated/rut5-base-paraphraser'
model_gen = T5ForConditionalGeneration.from_pretrained(model_name_gen)
tokenizer = T5Tokenizer.from_pretrained(model_name_gen)
model_gen.cuda();
model_gen.eval();

Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

In [10]:
def get_paraphrase(text, beams=5, grams=4, do_sample=False):

    x = tokenizer(text, return_tensors='pt', padding=True).to(model_gen.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model_gen.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [11]:
print(get_paraphrase('Сегодня на улице хорошая погода.'))

Сегодня погода на улице очень хорошая.
